In [1]:
import os, xml.etree.ElementTree as ET, numpy as np

In [2]:
cat readme.txt

This folder contains a sample of the data that will be released as a part 
of the two 2014 i2b2 challenge tracks. 

Each track has its own folder of sample data; the contents are outlined 
below.

----------------------------------------------

Track 1: De-identification 

Folder contents:

PHI/
	This folder contains only PHI-related annotations for the text 
in each document.  These files are the gold standard for the de-id track.

De-identification_guidelines_2014_distribution.pdf
	This file contains the guidelines for PHI identification

de-idi2b2_distribution.dtd
	This file is the DTD that describes the valid tags for the 
de-identification annotation task.

---------------------------------------------------

Track 2: Identifying risk factors for heart disease over time

Folder contents:

gold/
	This folder contains the gold standard annotations that will be used for
evaluation. These are document-level tags which are defined in the annotation
guidelines. Valid tags and attributes

In [3]:
START_CDATA = "<TEXT><![CDATA["
END_CDATA   = "]]></TEXT>"

TAGS        = ['MEDICATION', 'OBSEE', 'SMOKER', 'HYPERTENSION', 'PHI', 'FAMILY_HIST']

def read_xml_file(xml_path, PHI_tag_type='ALL_CHILDREN', match_text=True):
    with open(xml_path, mode='r') as f:
        lines = f.readlines()
        text, in_text = [], False
        for i, l in enumerate(lines):
            if START_CDATA in l:
                text.append(list(l[l.find(START_CDATA) + len(START_CDATA):]))
                in_text = True
            elif END_CDATA in l:
                text.append(list(l[:l.find(END_CDATA)]))
                break
            elif in_text:
                if xml_path.endswith('180-03.xml') and '0808' in l and 'Effingham' in l:
                    print("Adjusting known error")
                    l = l[:9] + ' ' * 4 + l[9:]
#                 elif xml_path.endswith('188-05.xml') and 'Johnson & Johnson' in l:
#                     print("Adjusting known error")
#                     l = l.replace('&', 'and')
                text.append(list(l))
        
    pos_transformer = {}
    
    linear_pos = 1
    for line, sentence in enumerate(text):
        for char_pos, char in enumerate(sentence):
            pos_transformer[linear_pos] = (line, char_pos)
            linear_pos += 1
        
    xml_parsed = ET.parse(xml_path)
    tag_containers = xml_parsed.findall('TAGS')
    assert len(tag_containers) == 1, "Found multiple tag sets!"
    tag_container = tag_containers[0]
    
    PHI_tags = tag_container.getchildren() if PHI_tag_type == 'ALL_CHILDREN' else tag_container.findall('PHI')
    PHI_labels = [['O'] * len(sentence) for sentence in text]
    for PHI_tag in PHI_tags:
        base_label = PHI_tag.attrib['TYPE']
        start_pos, end_pos, PHI_text = PHI_tag.attrib['start'], PHI_tag.attrib['end'], PHI_tag.attrib['text']
        start_pos, end_pos = int(start_pos)+1, int(end_pos)
        PHI_text = ' '.join(PHI_text.split())
#         if PHI_text == "0808 O’neil’s Court":
#             print("Adjusting known error")
#             end_pos -= 4
        if PHI_text == 'Johnson and Johnson' and xml_path.endswith('188-05.xml'):
            print("Adjusting known error")
            PHI_text = 'Johnson & Johnson'
        

        (start_line, start_char), (end_line, end_char) = pos_transformer[start_pos], pos_transformer[end_pos]
            
        obs_text = []
        for line in range(start_line, end_line+1):
            t = text[line]
            s = start_char if line == start_line else 0
            e = end_char if line == end_line else len(t)
            obs_text.append(''.join(t[s:e+1]).strip())
        obs_text = ' '.join(obs_text)
        obs_text = ' '.join(obs_text.split())
              
        if match_text: assert obs_text == PHI_text, (
            ("Texts don't match! %s v %s" % (PHI_text, obs_text)) + '\n' + str((
                start_pos, end_pos, line, s, e, t, xml_path
            ))
        )
        
        PHI_labels[end_line][end_char]     = 'I-%s' % base_label
        PHI_labels[start_line][start_char] = 'B-%s' % base_label
        
        for line in range(start_line, end_line+1):
            t = text[line]
            s = start_char+1 if line == start_line else 0
            e = end_char-1 if line == end_line else len(t)-1
            for i in range(s, e+1): PHI_labels[line][i] = 'I-%s' % base_label

    return text, PHI_labels
    
def merge_into_words(text_by_char, all_labels_by_char):
    assert len(text_by_char) == len(all_labels_by_char), "Incorrect # of sentences!"
    
    N = len(text_by_char)
    
    text_by_word, all_labels_by_word = [], []
    
    for sentence_num in range(N):
        sentence_by_char = text_by_char[sentence_num]
        labels_by_char   = all_labels_by_char[sentence_num]
        
        assert len(sentence_by_char) == len(labels_by_char), "Incorrect # of chars in sentence!"
        S = len(sentence_by_char)
        
        if labels_by_char == (['O'] * len(sentence_by_char)):
            sentence_by_word = ''.join(sentence_by_char).split()
            labels_by_word   = ['O'] * len(sentence_by_word)
        else: 
            sentence_by_word, labels_by_word = [], []
            text_chunks, labels_chunks = [], []
            s = 0
            for i in range(S):
                if i == S-1:
                    text_chunks.append(sentence_by_char[s:])
                    labels_chunks.append(labels_by_char[s:])
                elif labels_by_char[i] == 'O': continue
                else:
                    if i > 0 and labels_by_char[i-1] == 'O':
                        text_chunks.append(sentence_by_char[s:i])
                        labels_chunks.append(labels_by_char[s:i])
                        s = i
                    if labels_by_char[i+1] == 'O' or labels_by_char[i+1][2:] != labels_by_char[i][2:]:
                        text_chunks.append(sentence_by_char[s:i+1])
                        labels_chunks.append(labels_by_char[s:i+1])
                        s = i+1
                
            for text_chunk, labels_chunk in zip(text_chunks, labels_chunks):
                assert len(text_chunk) == len(labels_chunk), "Bad Chunking (len)"
                assert len(text_chunk) > 0, "Bad chunking (len 0)" + str(text_chunks) + str(labels_chunks)
                
                labels_set = set(labels_chunk)
                assert labels_set == set(['O']) or (len(labels_set) <= 3 and 'O' not in labels_set), (
                    ("Bad chunking (contents) %s" % ', '.join(labels_set))+ str(text_chunks) + str(labels_chunks)
                )
                
                text_chunk_by_word = ''.join(text_chunk).split()
                W = len(text_chunk_by_word)
                if W == 0: 
#                     assert labels_set == set(['O']), "0-word chunking and non-0 label!" + str(
#                         text_chunks) + str(labels_chunks
#                     )
                    continue
                
                if labels_chunk[0] == 'O': labels_chunk_by_word = ['O'] * W
                elif W == 1:               labels_chunk_by_word = [labels_chunk[0]]
                elif W == 2:               labels_chunk_by_word = [labels_chunk[0], labels_chunk[-1]]
                else:                      labels_chunk_by_word = [
                        labels_chunk[0]
                    ] + [labels_chunk[1]] * (W - 2) + [
                        labels_chunk[-1]
                    ]
                    
                sentence_by_word.extend(text_chunk_by_word)
                labels_by_word.extend(labels_chunk_by_word)

        assert len(sentence_by_word) == len(labels_by_word), "Incorrect # of words in sentence!"    
        
        if len(sentence_by_word) == 0: continue
            
        text_by_word.append(sentence_by_word)
        all_labels_by_word.append(labels_by_word)
    return text_by_word, all_labels_by_word

def reprocess_PHI_labels(folders, base_path='.', PHI_tag_type='PHI', match_text=True, dev_set_size=None):
    all_texts_by_patient, all_labels_by_patient = {}, {}

    for folder in folders:
        folder_dir = os.path.join(base_path, folder)
        xml_filenames = [x for x in os.listdir(folder_dir) if x.endswith('xml')]
        for xml_filename in xml_filenames:
            patient_num = int(xml_filename[:3])
            xml_filepath = os.path.join(folder_dir, xml_filename)
            
            text_by_char, labels_by_char = read_xml_file(
                xml_filepath,
                PHI_tag_type=PHI_tag_type,
                match_text=match_text
            )
            text_by_word, labels_by_word = merge_into_words(text_by_char, labels_by_char)
            
            if patient_num not in all_texts_by_patient:
                all_texts_by_patient[patient_num] = []
                all_labels_by_patient[patient_num] = []
            
            all_texts_by_patient[patient_num].extend(text_by_word)
            all_labels_by_patient[patient_num].extend(labels_by_word)
            
    patients = set(all_texts_by_patient.keys())
    
    if dev_set_size is None: train_patients, dev_patients = list(patients), []
    else:
        N_train = int(len(patients) * (1-dev_set_size))
        patients_random = np.random.permutation(list(patients))
        train_patients = list(patients_random[:N_train])
        dev_patients   = list(patients_random[N_train:])
    
    train_texts, train_labels = [], []
    dev_texts, dev_labels = [], []
    
    for patient_num in train_patients:
        train_texts.extend(all_texts_by_patient[patient_num])
        train_labels.extend(all_labels_by_patient[patient_num])

    for patient_num in dev_patients:
        dev_texts.extend(all_texts_by_patient[patient_num])
        dev_labels.extend(all_labels_by_patient[patient_num])


    train_out_text_by_sentence = []
    for text, labels in zip(train_texts, train_labels):
        train_out_text_by_sentence.append('\n'.join('%s %s' % x for x in zip(text, labels)))
    dev_out_text_by_sentence = []
    for text, labels in zip(dev_texts, dev_labels):
        dev_out_text_by_sentence.append('\n'.join('%s %s' % x for x in zip(text, labels)))

    return '\n\n'.join(train_out_text_by_sentence), '\n\n'.join(dev_out_text_by_sentence)

In [4]:
final_train_text, final_dev_text = reprocess_PHI_labels(
    ['training-PHI-Gold-Set1/', 'training-PHI-Gold-Set2/'], PHI_tag_type='ALL_CHILDREN',
    dev_set_size=0.1, match_text=True
)

Adjusting known error
Adjusting known error


In [5]:
test_text, _ = reprocess_PHI_labels(
    ['testing-PHI-Gold-fixed'], PHI_tag_type='ALL_CHILDREN', match_text=False, dev_set_size=None
)

In [6]:
print(final_train_text[:500])

Record O
date: O
2067-11-30 B-DATE

CARDIOLOGY O

OCHILTREE B-HOSPITAL
GENERAL I-HOSPITAL
HOSPITAL I-HOSPITAL

Reason O
for O
visit: O

Follow-up O
appointment O
for O
cardiomyopathy O

Interval O
History: O

Mr O
Lara B-PATIENT
reports O
no O
problems O
since O
the O
time O
of O
his O
last O
visit. O
He O
insistists O
that O
he O
has O
absolutely O
abstained O
from O
etoh O
since O
his O
last O
visit. O

Denies O
fevers, O
chills. O
denies O
palpitations O
or O
syncope. O

Under O
a O
fair O
am


In [7]:
print(final_dev_text[:400])

Record O
date: O
2082-02-28 B-DATE

c.c. O
Preop O
PE O
for O
cataract O
extraction O
on O
3/7 B-DATE

S: O
ROS: O
General: O
no O
history O
of O
fatigue, O
weight O
change, O
loss O
of O
appetite, O
or O
weakness. O

HEENT: O
no O
history O
of O
head O
injury, O
glaucoma, O
tinnitus, O
vertigo, O
motion O
sickness, O
URI, O
hearing O
loss. O

Cardiovascular: O
+ O
hypertension O
- O
adequate O
co


In [8]:
print(test_text[:400])

Record O
date: O
2080-02-18 B-DATE

SDU O
JAR O
Admission O
Note O

Name: O
Yosef B-PATIENT
Villegas I-PATIENT

MR: O
8249813 B-MEDICALRECORD

DOA: O
2/17/80 B-DATE

PCP: O
Gilbert B-DOCTOR
Perez I-DOCTOR

Attending: O
YBARRA B-DOCTOR

CODE: O
FULL O

HPI: O
70 B-AGE
yo O
M O
with O
NIDDM O
admitted O
for O
cath O
after O
positive O
MIBI. O
Pt O
has O
had O
increasing O
CP O
and O
SOB O
on O
exert


In [10]:
labels = {}
for s in final_train_text, final_dev_text, test_text:
    for line in s.split('\n'):
        if line == '': continue
        label = line.split()[-1]
        assert label == 'O' or label.startswith('B-') or label.startswith('I-'), "label wrong! %s" % label
        if label not in labels: labels[label] = 1
        else: labels[label] += 1

In [11]:
labels

{'O': 777860,
 'B-DATE': 12459,
 'B-HOSPITAL': 2312,
 'I-HOSPITAL': 1835,
 'B-PATIENT': 2195,
 'B-AGE': 1997,
 'B-DOCTOR': 4797,
 'I-DOCTOR': 3482,
 'I-DATE': 1379,
 'B-COUNTRY': 183,
 'B-PROFESSION': 413,
 'I-PROFESSION': 346,
 'B-ORGANIZATION': 206,
 'I-ORGANIZATION': 173,
 'B-STREET': 352,
 'I-STREET': 717,
 'B-CITY': 654,
 'I-CITY': 171,
 'B-STATE': 504,
 'B-ZIP': 352,
 'I-PATIENT': 1192,
 'B-MEDICALRECORD': 1033,
 'I-MEDICALRECORD': 47,
 'B-IDNUM': 456,
 'B-USERNAME': 356,
 'B-PHONE': 524,
 'I-COUNTRY': 21,
 'I-PHONE': 100,
 'I-AGE': 10,
 'I-IDNUM': 30,
 'B-FAX': 10,
 'I-FAX': 2,
 'B-DEVICE': 15,
 'B-EMAIL': 5,
 'B-LOCATION-OTHER': 17,
 'I-LOCATION-OTHER': 15,
 'I-STATE': 18,
 'B-URL': 2,
 'I-URL': 4,
 'B-BIOID': 1,
 'B-HEALTHPLAN': 1,
 'I-HEALTHPLAN': 1,
 'I-DEVICE': 2}

In [28]:
with open('./processed/train.tsv', mode='w') as f:
    f.write(final_train_text)
with open('./processed/dev.tsv', mode='w') as f:
    f.write(final_dev_text)
with open('./processed/test.tsv', mode='w') as f:
    f.write(test_text)